In [76]:
from re import sub, compile
from requests import get, exceptions
from pandas import DataFrame, set_option
import numpy as np
import seaborn as sns
from pylab import rcParams, savefig
import matplotlib
from gensim.utils import tokenize
from gensim.models import doc2vec
from nltk import sent_tokenize
from itertools import chain
from nltk.tokenize import wordpunct_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import models, corpora
from pymorphy2 import MorphAnalyzer
from pickle import load
from collections import namedtuple
morph = MorphAnalyzer()

Утилсы: регулярки, загрузка файлов и т.д.

In [8]:
def cut(data):
    r = compile(r'<.*?>|>>.*|&#(47|92);|&quot;|&gt;|(http|https):.*')
    return r.sub('', data)

def punctuate(data):
    r = compile( r'([a-zA-Z])([,.!])', r'\1 \2')
    return r.sub('', data)

def pos_cut(s):
    s = sub(r"ADJF", "ADJ", s)
    s = sub(r"ADVB", "ADV", s)
    s = sub(r"INFN", "VERB", s)
    return s

def topic_cut(word):
    return sub('[^a-zA-Zа-яА-Я]', '', word)

def remove_noise(s):
    remove_noise = compile('(@[A-Za-z_0-9]+)|\[.+]\,|\[.+]|([^0-9A-Za-zА-Яа-я[\.’,?!-:] \t])'
                           '|(\w+:\/\/\S+)')
    return (sub(remove_noise, '',  s))

def check_for_rus_lan(word):
    regex = compile('[А-Яа-я]+$')
    if regex.match(word):
        return True
    return False

In [31]:
documents = load(open('pickle/pr.pickle', 'rb'))

Загрузка токенизатора, токенизация данных

In [ ]:
stop = stopwords.words('russian')
alpha_tokenizer = RegexpTokenizer('[A-Za-zА-Яа-я]\w+')

def my_tokenize(sent):
    sents = list(alpha_tokenizer.tokenize(sent))
    return set([morph.parse(sent.lower())[0].normal_form for sent in sents if sent.lower() not in stop and len(sent.lower()) > 1])

model_data = [my_tokenize(sent) for sent in documents]

Инициализация стоп-слов

In [ ]:
stop = stopwords.words('russian')
stop += stopwords.words('english')
my_stop = ['весь', 'всякий', 'тот', 'это', 'самое', 'каким', 'сама', 'никак', 'она', 'она', 'оно', 'какие', 'какого', 'которая', 'многое', 'чему', 'всему', 'раза', 'сразу', 'весь', 'раз', 'пор', 'например', 'вроде', 'которые', 'который', 'просто', 'очень', 'почему', 'вообще', 'ещё', 'типа', 'ради', 'всё', 'хотя', 'тебе', 'нужно', 'пока']
my_stop_verb = ['сделать', 'можешь', 'могу', 'могут', 'делать', 'будешь', 'быть', 'будут', 'смочь']
[stop.append(morph.parse(i)[0].normal_form) for i in my_stop]
[stop.append(morph.parse(i)[0].normal_form) for i in my_stop_verb]
dvach_stop = ['такое', 'лол', 'бля']
stop += dvach_stop

Word2Vec-модель

In [ ]:
min_count = 5
size = 800
window = 20

model = models.Word2Vec(model_data, min_count=min_count, size=size, window=window,  workers=4)

In [ ]:
word = morph.parse('кодить')[0].normal_form
print(word)
keyword = morph.parse('программирование')[0].normal_form

model.similarity(word, keyword)

In [ ]:
result = []

for test_word in documents[10000:10050]:
    amount = 0
    similarity = 0
    for i in wordpunct_tokenize(test_word):
        sim = 0
        try:
            sim = model.similarity(morph.parse(i)[0].normal_form, keyword)
            amount += 1
        except KeyError:
            pass
        similarity += sim
    positive = False
    sim_am = 0
    try:
        sim_am = similarity/amount
    except ZeroDivisionError:
        pass
    if sim_am > 0.5:
        positive = True
    result.append({'text': test_word, 'similarity': sim_am, 'positive': positive})

In [ ]:
pd.set_option('display.max_colwidth', 200)
DataFrame.from_records(result)

Получение TF-IDF весов для корпуса и инициализация словаря, в котором каждому слову ассоциирован вес

In [ ]:
corpus = documents
vectorizer = TfidfVectorizer(min_df=1)
X = vectorizer.fit_transform(corpus)
idf = vectorizer.idf_
weights_dict = dict(zip(vectorizer.get_feature_names(), idf))

Проверка работы TF-IDF модели

In [ ]:
result = []

for test_word in documents[10000:10050]:
    amount = 0
    similarity = 0
    for i in wordpunct_tokenize(test_word):
        sim = 0
        try:
            sim = model.similarity(morph.parse(i)[0].normal_form, keyword)
            amount += 1
            try:
                weight = round(weights_dict[i], 2)
                sim *= weight
            except:
                pass
        except KeyError:
            pass
        similarity += sim
    positive = False
    sim_am = 0
    try:
        sim_am = similarity/amount
    except ZeroDivisionError:
        pass
    if sim_am > 0.5:
        positive = True
    result.append({'text': test_word, 'similarity': sim_am, 'positive': positive})

In [77]:
docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i, text in enumerate(df.text):
    words = text.lower().split()
    tags = [i]
    docs.append(analyzedDocument(words, tags))

In [79]:
model = doc2vec.Doc2Vec(docs, size = 100, window = 5, min_count = 10, workers=4)